In [244]:
from bs4 import BeautifulSoup 
import requests
import pickle 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
import re

In [6]:
# In place of running this and making a web request, import the pickle file from below
# Assign URL 
#url = "https://asn.flightsafety.org/database/engines/"

# Make a GET request to fetch the raw HTML content 
#html_content = requests.get(url).text 

# Parse the response with html.parser
#soup = BeautifulSoup(html_content,"html.parser") 


In [246]:
# The data we need are wrapped in <a> tags. Grab all the <a> tags here and we'll filter later.
datas = soup.find_all("a")

In [9]:
#Make list of URLs and list oF Aircraft to create dictionary
k = []
v = []
for item in datas:
    if 'engine' in item['href']:
        k.append(item['href'])
        v.append(item.text)

lookup_dict = {'URL': k, 'Engine': v}

In [10]:
url_df = pd.DataFrame.from_dict(lookup_dict)

In [11]:
with open('lookup_dict.pkl', 'wb') as f:
    pickle.dump(lookup_dict, f)

In [ ]:
#with open('zippedData/lookup_dict.pkl', 'rb') as f:
    #lookup_dict = pickle.load(f)

In [12]:
# Take in the engine and retriev the matching data page with Aircraft data
# Add Aircraft list from web a python list.
# Return the python list
url_p1 = 'https://asn.flightsafety.org'
def lookup_ac(url_p2):
    return_list = []
    url = f'{url_p1}{url_p2}'
    sub_content = requests.get(url).text
    soup2 = BeautifulSoup(sub_content,"html.parser")
    u_list=soup2.find_all('ul')[1]
    i = 0 
    while i < len(u_list.select('li')):
        (u_list.select('li')[i].text)
        return_list.append(u_list.select('li')[i].text)
        i += 1
    return(return_list)

In [13]:
# Create Aircraft column to list aircraft for each engine.
url_df['Aircraft'] = url_df['URL'].map(lookup_ac)

In [15]:
# Save df to pickle file.
url_df.to_pickle('zippedData/url_df.pkl')

In [16]:
# Create a row for each aircraft for each engine. 
# Iterate through url_df['Aircraft'] column. For each element
# in list, create a new row.

engines = []
aircraft = []
for idx, row in url_df.iterrows():
    for ac in row['Aircraft']:
        engines.append(row['Engine'])
        aircraft.append(ac)
        


In [17]:
# Create dictionary from lists to load into dataFrame
ac_eng_dict = {'Engine': engines, 'Aircraft': aircraft}

In [18]:
# Create dataFrame from dictionary
engines_df = pd.DataFrame.from_dict(ac_eng_dict)

In [20]:
# Clean up aircraft names
engines_df['Aircraft'].replace('-', '', inplace=True, regex=True)
engines_df['Aircraft'].replace('/', '', inplace=True, regex=True)
engines_df.drop_duplicates(inplace=True)

In [29]:
''' This function will 
1) takes in a airplane name
2) matches the cloest short name
3) returns the match
'''

def get_short_name(model):
    m_string = model.split(' ')
    if len(m_string) == 1:
        return(m_string[0])
    else:
        return(m_string[1])
    

In [ ]:
''' 
This function will 
1) take in a model string from bts_map_df 
2) match it with a newmodel from engines_df
3) update engines_df with new_model column
TODO: Replace with faster version.
'''
def match_types(model):
    for newmodel in engines_df['Model']:
        if model.startswith(newmodel):
            #print(f'Processing: {model} \t:{newmodel}')
            return newmodel

In [23]:
# Read data from Aircraft_Safety dataset
bts_map_df = pd.read_excel('zippedData/bts_actype.2.xlsx')


In [24]:
bts_map_df

,Unnamed: 0,Description,Model
0,0,Boeing 757-200,757
1,72814,Pilatus PC-12,PC12
2,98137,Cessna Citation X Model 650/550B/550XL,CitationX
3,99813,Boeing 737-400,737
4,120190,Boeing 737-300,737
...,...,...,...
182,2018715,Boeing 747-200/300,747
183,2018731,Cessna 180A/B,180
184,2018733,Gulfstream Aerospace G500C,G500
185,2018734,Gulfstream Aerospace Gulfstream G600,G500


In [26]:
# Fill in the Model column with short names
engines_df['Model'] = [get_short_name(model) for model in engines_df['Aircraft']]

In [27]:
engines_df.columns

Index(['Engine', 'Aircraft', 'Model'], dtype='object')

In [327]:
engines_df[0:50]


,Engine,Aircraft,Model
0,Allison 250,BrittenNorman BN2 Islander,BN2
1,Allison 250,GAF Nomad,Nomad
2,Allison 501,Aero Spacelines Mini Guppy Turbine,Spacelines
3,Allison 501,Convair CV580,CV580
4,Allison 501,Lockheed C130 Hercules,C130
5,Allison 501,Lockheed L100 Hercules,L100
6,Allison 501,Lockheed L188 Electra,L188
7,Allison 501,Lockheed P3 Orion,P3
8,Allison AE2100,Lockheed C130 Hercules,C130
9,Allison AE2100,SAAB 2000,2000


In [30]:
# Save df to pickle file.
#engines_df.to_pickle('zippedData/engines_df.pkl')

In [297]:
wiki_lis = pd.read_html('zippedData/Wiki.html')

In [298]:
com_df = wiki_lis[1]
reg_df = wiki_lis[2]
srt_df = wiki_lis[3]
med_df = wiki_lis[4]
lng_df = wiki_lis[5]

In [299]:
srt_df.rename(columns={'Fuel Burn': 'Fuel burn'}, inplace=True)
srt_df.rename(columns={'Fuel efficiency per seat': 'Fuel per seat'}, inplace=True)

In [300]:
reg_df.rename(columns={'Fuel efficiency per seat': 'Fuel per seat'}, inplace=True)

In [301]:
reg_df.columns

Index(['Model', 'First flight', 'Seats', 'Sector', 'Fuel burn',
       'Fuel per seat'],
      dtype='object')

In [302]:
wiki_lis[1]['Fuel burn']

0      4.23 kg/km (15.0 lb/mi)
1      4.34 kg/km (15.4 lb/mi)
2       1.26 kg/km (4.5 lb/mi)
3       1.67 kg/km (5.9 lb/mi)
4       1.42 kg/km (5.0 lb/mi)
5       1.56 kg/km (5.5 lb/mi)
6      1.00 kg/km (3.56 lb/mi)
7      2.21 kg/km (7.83 lb/mi)
8      2.18 kg/km (7.73 lb/mi)
9     2.95 kg/km (10.47 lb/mi)
10    3.47 kg/km (12.31 lb/mi)
11      2.16 kg/km (7.7 lb/mi)
12      0.94 kg/km (3.3 lb/mi)
13      1.22 kg/km (4.3 lb/mi)
14      0.92 kg/km (3.3 lb/mi)
15     1.64 kg/km (5.83 lb/mi)
16     1.76 kg/km (6.23 lb/mi)
17       1.1 kg/km (3.9 lb/mi)
18      1.75 kg/km (6.2 lb/mi)
Name: Fuel burn, dtype: object

In [303]:
def clean_fuel_seat(str):
    #print(str)
    r = re.search('([0-9]+.[0-9]+)\\xa0L', str)
    rg = r.group(1)
    return float(rg)
    

In [304]:
def clean_fuel_burn(str):
    #print(str)
    #r = re.search('(([0-9].[0-9]+)|([0-9]+))\\xa0kg', str)
    r = re.search('^(.*?)\\xa0kg', str)
    rg = (r.group(1))
    #print(rg)
    return float(rg)
    

In [305]:
clean_fuel_burn(reg_df['Fuel burn'][8])

2.89

In [306]:
clean_fuel_burn(med_df['Fuel burn'][6])

6.81

In [317]:
lng_df['Fuel burn'] = lng_df['Fuel burn'].apply(clean_fuel_burn)

In [318]:
lng_df['Fuel per seat'] = lng_df['Fuel per seat'].apply(clean_fuel_seat)

In [307]:
com_df['Fuel burn'] = com_df['Fuel burn'].apply(clean_fuel_burn)

In [319]:
lng_df

,Model,First flight,Seats,Sector,Fuel burn,Fuel per seat
0,Airbus A330-200,1997,241,"6,000 nmi (11,000 km)",6.40,3.32
1,Airbus A330-200,1997,248,"5,549 nmi (10,277 km)",6.55,3.30
2,Airbus A330-300,1992,274,"5,548 nmi (10,275 km)",6.81,3.11
3,Airbus A330-800,2017,248,"4,650 nmi (8,610 km)",5.45,2.75
4,Airbus A330-900,2017,300,"4,650 nmi (8,610 km)",5.94,2.48
5,Airbus A340-300,1992,262,"6,000 nmi (11,000 km)",7.32,3.49
6,Airbus A350-900,2013,315,"4,972 nmi (9,208 km)",6.03,2.39
7,Airbus A350-900,2013,318,"5,534 nmi (10,249 km)",6.52,2.56
8,Airbus A350-900,2013,315,"6,542 nmi (12,116 km)",7.07,2.81
9,Airbus A350-1000,2016,327,"5,531 nmi (10,243 km)",7.46,2.85


In [308]:
com_df['Fuel per seat'] = com_df['Fuel per seat'].apply(clean_fuel_seat)

In [309]:
reg_df['Fuel burn'] = reg_df['Fuel burn'].apply(clean_fuel_burn)

In [310]:
reg_df['Fuel per seat'] = reg_df['Fuel per seat'].apply(clean_fuel_seat)

In [311]:
srt_df['Fuel burn'] = srt_df['Fuel burn'].apply(clean_fuel_burn)

In [312]:
srt_df['Fuel per seat'] = srt_df['Fuel per seat'].apply(clean_fuel_seat)

In [314]:
med_df['Fuel burn'] = med_df['Fuel burn'].apply(clean_fuel_burn)

In [315]:
med_df['Fuel per seat'] = med_df['Fuel per seat'].apply(clean_fuel_seat)

In [274]:
lng_df

,Model,First flight,Seats,Sector,Fuel burn,Fuel per seat
0,Airbus A330-200,1997,241,"6,000 nmi (11,000 km)",6.40,3.32 L/100 km (71 mpg‑US)[98]
1,Airbus A330-200,1997,248,"5,549 nmi (10,277 km)",6.55,3.3 L/100 km (71 mpg‑US)[107]
2,Airbus A330-300,1992,274,"5,548 nmi (10,275 km)",6.81,3.11 L/100 km (76 mpg‑US)[107]
3,Airbus A330-800,2017,248,"4,650 nmi (8,610 km)",5.45,2.75 L/100 km (86 mpg‑US)[108]
4,Airbus A330-900,2017,300,"4,650 nmi (8,610 km)",5.94,2.48 L/100 km (95 mpg‑US)[108]
5,Airbus A340-300,1992,262,"6,000 nmi (11,000 km)",7.32,3.49 L/100 km (67.4 mpg‑US)[98]
6,Airbus A350-900,2013,315,"4,972 nmi (9,208 km)",6.03,2.39 L/100 km (98 mpg‑US)[99]
7,Airbus A350-900,2013,318,"5,534 nmi (10,249 km)",6.52,2.56 L/100 km (92 mpg‑US)[107]
8,Airbus A350-900,2013,315,"6,542 nmi (12,116 km)",7.07,2.81 L/100 km (84 mpg‑US)[109]
9,Airbus A350-1000,2016,327,"5,531 nmi (10,243 km)",7.46,2.85 L/100 km (83 mpg‑US)[107]
